## Deezer playlist to csv

In [1]:
import requests
import pandas as pd

playlist_id_list = ["13301519863",
                    "11109924384",
                    "11109924124",
                    "11109918904",
                    "11109919544",
                    "11917580341",
                    "11109924924",
                    "11109923824",
                    "11912452141",
                    "11109920664",
                    "11537618704",
                    "11164434904",
                    "12126532871",
                    "11109921284",
                    "11917626441",
                    "11959053381",
                    "11109921064",
                    ]

df_tracks = pd.DataFrame(columns=["id", "title", "album", "duration", "release_date", "rank", "bpm", "gain"])

for playlist_id in playlist_id_list:
    response = requests.get(f"https://api.deezer.com/playlist/{playlist_id}")
    playlist = response.json()
    playlist_name = f'in_{playlist["title"].replace(" ", "_")}'

    track_id_list = [playlist["tracks"]["data"][i]["id"] for i in range(len(playlist["tracks"]["data"]))]

    for track_id in track_id_list:
        if track_id not in df_tracks["id"].values:
            response = requests.get(f"https://api.deezer.com/track/{track_id}")
            track = response.json()
            artists = ([artist["name"] for artist in track["contributors"]])
            df_tracks = pd.concat([df_tracks, pd.DataFrame([{
                "id": track["id"],
                "title": track["title"],
                "album": track["album"]["title"],
                "duration": track["duration"],
                "release_date": track["album"]["release_date"],
                "rank": track["rank"],
                "bpm": track["bpm"],
                "gain": track["gain"],
                f"{playlist_name}": True
            }])], ignore_index=True)

            for i in range(len(artists)):
                df_tracks.loc[df_tracks.index[-1], f'artist_{i+1}'] = artists[i] if i < len(artists) else None

        else:
            df_tracks.loc[df_tracks["id"] == track_id, f'{playlist_name}'] = True


artist_columns = [col for col in df_tracks.columns if col.startswith("artist")]
in_columns = [col for col in df_tracks.columns if col.startswith("in_")]
other_columns = [col for col in df_tracks.columns if col not in artist_columns and col not in in_columns]
df_tracks = df_tracks[other_columns + artist_columns + in_columns]
       
df_tracks.head()

,id,title,album,duration,release_date,rank,bpm,gain,artist_1,artist_2,...,in_VOCAR2D2,in_chill_vibes,in_popito,in_e-fav,in_popita,in_city_vibes,in_musical,in_sugoma,in_happy_era,in_the_K
0,2817563312,Playground (from the series Arcane League of L...,Arcane League of Legends (Soundtrack from the ...,230,2021-11-20,790810,0,-9.4,Bea Miller,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2817563342,Dirty Little Animals (from the series Arcane L...,Arcane League of Legends (Soundtrack from the ...,205,2021-11-20,751516,0,-7.6,BONES UK,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2817563352,Enemy (from the series Arcane League of Legends),Arcane League of Legends (Soundtrack from the ...,173,2021-11-20,712534,0,-8.5,Imagine Dragons,JID,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2817563362,Guns for Hire (from the series Arcane League o...,Arcane League of Legends (Soundtrack from the ...,226,2021-11-20,824232,0,-9.4,Woodkid,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2817563372,Misfit Toys (from the series Arcane League of ...,Arcane League of Legends (Soundtrack from the ...,189,2021-11-20,711160,0,-7.8,Pusha T,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
duplicate_titles = df_tracks[df_tracks.duplicated(subset='title', keep=False)]
duplicate_titles

,id,title,album,duration,release_date,rank,bpm,gain,artist_1,artist_2,...,in_VOCAR2D2,in_chill_vibes,in_popito,in_e-fav,in_popita,in_city_vibes,in_musical,in_sugoma,in_happy_era,in_the_K
0,2817563312,Playground (from the series Arcane League of L...,Arcane League of Legends (Soundtrack from the ...,230,2021-11-20,790810,0,-9.4,Bea Miller,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2817563342,Dirty Little Animals (from the series Arcane L...,Arcane League of Legends (Soundtrack from the ...,205,2021-11-20,751516,0,-7.6,BONES UK,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2817563362,Guns for Hire (from the series Arcane League o...,Arcane League of Legends (Soundtrack from the ...,226,2021-11-20,824232,0,-9.4,Woodkid,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2817563372,Misfit Toys (from the series Arcane League of ...,Arcane League of Legends (Soundtrack from the ...,189,2021-11-20,711160,0,-7.8,Pusha T,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2817563382,Dynasties and Dystopia (from the series Arcane...,Arcane League of Legends (Soundtrack from the ...,178,2021-11-20,825330,0,-8.1,Denzel Curry,GIZZLE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2817563392,Snakes (from the series Arcane League of Legends),Arcane League of Legends (Soundtrack from the ...,161,2021-11-20,758850,0,-8,PVRIS,MIYAVI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,1544617202,Dynasties and Dystopia (from the series Arcane...,Arcane League of Legends (Soundtrack from the ...,178,2021-11-20,71015,0,-8.1,Denzel Curry,GIZZLE,...,NaN,NaN,True,True,NaN,NaN,NaN,NaN,NaN,NaN
45,593087402,The Other Side,The Greatest Showman: Reimagined (Deluxe),210,2017-12-08,245138,153.69,-8.5,Max,Ty Dolla $ign,...,NaN,NaN,True,True,NaN,NaN,NaN,NaN,NaN,NaN
46,593087362,The Greatest Show,The Greatest Showman: Reimagined (Deluxe),174,2017-12-08,264767,154.27,-7.6,Panic! At the Disco,NaN,...,NaN,NaN,True,True,NaN,NaN,NaN,NaN,NaN,NaN
47,1544617192,Misfit Toys (from the series Arcane League of ...,Arcane League of Legends (Soundtrack from the ...,189,2021-11-20,66797,0,-7.8,Pusha T,Mako,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
=((B2*0,1)*1+(C2*0,285))

In [8]:
all_artists = df_tracks[artist_columns].stack().reset_index(level=1, drop=True).to_frame('artist')
all_artists['in_fav'] = df_tracks.loc[all_artists.index, 'in_fav'].values

artist_counts = all_artists['artist'].value_counts().to_frame('nombre_de_titres')
fav_counts = all_artists[all_artists['in_fav'] == True]['artist'].value_counts().to_frame('nombre_de_favoris')

artist_summary = artist_counts.join(fav_counts, how='left').fillna(0)
artist_summary['coef'] = round((artist_summary['nombre_de_titres'] * 0.1) * (1 + artist_summary['nombre_de_favoris'] * 0.25), 3)
artist_summary.reset_index(inplace=True)
artist_summary.rename(columns={'index': 'artiste'}, inplace=True)

artist_summary

,artiste,nombre_de_titres,nombre_de_favoris,coef
0,milet,25,10.0,8.750
1,League of Legends,21,7.0,5.775
2,MIYAVI,18,5.0,4.050
3,Imagine Dragons,16,5.0,3.600
4,Shayfer James,14,6.0,3.500
...,...,...,...,...
473,Iamalex,1,0.0,0.100
474,Sleepy Fish,1,0.0,0.100
475,Post Malone,1,0.0,0.100
476,Morgan Wallen,1,0.0,0.100
